# Face Mask Detection
In this project, I will use VGG16 and Haar Cascade classifier to detect whether an individual is wearing a face mask or not using live webcam.

### Data Preprocessing

In [24]:
# Importing the dependencies
import os
from tensorflow.keras.preprocessing import image
import cv2
import random
import numpy as np

In [3]:
# Lets set the categories
categories = ["with_mask", "without_mask"]

In [4]:
# Lets set the input path, load the images, read the images, resize the images, and store the category and class to a list 
data = [] 
for category in categories:
    path = os.path.join("train", category)
    
    label = categories.index(category)
    
    for file in os.listdir(path):
        img_path = os.path.join(path,file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224,224))
        
        data.append([img,label])

In [5]:
# Lets shuffle the data so that the model does not become biased
random.shuffle(data)

In [6]:
# Lets seperate the feature and labels
X = []
y = []

for features, label in data:
    X.append(features)
    y.append(label)

In [7]:
# Lets convert the lists to dataframabse
X = np.array(X)
y = np.array(y)

In [8]:
# Lets see the shape of X
X.shape

(814, 224, 224, 3)

In [9]:
# Lets see the shape of y
y.shape

(814,)

In [10]:
# Feature scaling
X = X/255

In [11]:
# Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [12]:
# Lets see the shapes
print(X_train.shape)
print(X_test.shape)

(651, 224, 224, 3)
(163, 224, 224, 3)


### Building the VGG16 model

In [13]:
# Lets load the VGG model
from tensorflow.keras.applications.vgg16 import VGG16
vgg = VGG16()

In [14]:
# Lets see the VGG16 model summary
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [15]:
# Removing the last layer
from tensorflow.keras import Sequential
model = Sequential()
for layer in vgg.layers[:-1]:
    model.add(layer)

In [16]:
# Lets see the model summary now
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [17]:
# We will not train the layers
for layer in model.layers:
    layer.trainable = False

In [18]:
# Lets see the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [19]:
# Adding the last layer
from tensorflow.keras.layers import Dense
model.add(Dense(1, activation = "sigmoid"))

In [20]:
# Lets see the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [21]:
# Lets compile the model
model.compile(optimizer = "Adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [23]:
# Model training
model.fit(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Epoch 1/5
21/21 [==============================] - 1897s 86s/step - loss: 0.7303 - accuracy: 0.5346 - val_loss: 0.5827 - val_accuracy: 0.5460
Epoch 2/5
21/21 [==============================] - 1929s 94s/step - loss: 0.4845 - accuracy: 0.7696 - val_loss: 0.3466 - val_accuracy: 0.9387
Epoch 3/5
21/21 [==============================] - 2244s 108s/step - loss: 0.3485 - accuracy: 0.9155 - val_loss: 0.2866 - val_accuracy: 0.9387
Epoch 4/5
21/21 [==============================] - 7945s 394s/step - loss: 0.2803 - accuracy: 0.9309 - val_loss: 0.2255 - val_accuracy: 0.9509
Epoch 5/5
21/21 [==============================] - 1884s 91s/step - loss: 0.2395 - accuracy: 0.9401 - val_loss: 0.1995 - val_accuracy: 0.9509


In [60]:
# Saving the model to reuse again
model.save("model_maskdetection.h5")

In [34]:
# Testing
sample1 = cv2.imread("masks2.0/test/0/1.jpg")
sample1 = cv2.resize(sample1, (224,224))
detect_face_mask(sample1)

C:\Users\SWARNAVA\anaconda3\envs\cv\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[1]])

Model predicts correctly!

In [36]:
# Testing
sample1 = cv2.imread("masks2.0/test/1/1.jpeg")
sample1 = cv2.resize(sample1, (224,224))
detect_face_mask(sample1)

C:\Users\SWARNAVA\anaconda3\envs\cv\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[0]])

Model is able to classify classes properly!

### Face mask Detection model

In [59]:
# Lets use cv2 to capture live stream webcam video and detect mask or no mask
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    # Calling the detection method
    img = cv2.resize(frame, (224,224))
    
    y_pred = detect_face_mask(img)
    
    coods = detect_face(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
    
    for x,y,w,h in coods:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
    
    if y_pred == 0:
        draw_label(frame, "Mask", (30,30), (0,255,0))
    else:
        draw_label(frame, "No Mask", (30,30), (0,0,255))
    
    cv2.imshow("window", frame)
    
    if cv2.waitKey(1) & 0xFF == ord("x"):
        break

cv2.destroyAllWindows()

In [27]:
# Lets create a function for detecting face mask
def detect_face_mask(img):
    y_pred = model.predict_classes(img.reshape(1,224,224,3))
    return y_pred

In [41]:
# Lets create a function for displaying text
def draw_label(img, text, pos, bg_color):
    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, cv2.FILLED)
    
    end_x = pos[0] + text_size[0][0] + 2
    end_y = pos[0] + text_size[0][1] -2
    
    cv2.rectangle(img, pos, (end_x,end_y), bg_color, cv2.FILLED)
    cv2.putText(img, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1, cv2.LINE_AA)

In [53]:
# Lets import the haar cascade classifier
haar = cv2.CascadeClassifier("Haar Cascade\haarcascade_frontalface_default.xml")

# Lets create a function for detecting face
def detect_face(img):
    coods = haar.detectMultiScale(img)
    return coods

In [56]:
# Face Detection
while True:
    ret, frame = cap.read()
    
    coods = detect_face(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
    
    for x,y,w,h in coods:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
        
    cv2.imshow("window", frame)
    
    if cv2.waitKey(1) & 0xFF == ord("x"):
        break
        
cv2.destroyAllWindows()